In [12]:
!pip install PyMuPDF

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
import fitz
pdf_document = fitz.open("1_resume_job1_ChatGPT_0.pdf")
pdf_document_1 = fitz.open("8_resume_job3_Gemini_0.docx")

for page_number in range(len(pdf_document_1)):
    page = pdf_document_1[page_number]  # Get a page
    text = page.get_text()  # Extract text


    if page_number == 0:
        text = replace_names(text)
    text = replace_dates(text)
    text = replace_phone_numbers(text)
    text = replace_addresses(text)
    text = replace_companies(text)
    text = replace_universities(text)
    text = replace_links(text)
    text = replace_emails(text)
    print(f"Page {page_number + 1}:\n{text}\n" + "-"*50)



NameError: name 'replace_names' is not defined

# Code to Double Check which words get replaced

In [6]:
import fitz
import re
from difflib import ndiff  # To show differences between original and modified text

pdf_document_1 = fitz.open("1_resume_job1_ChatGPT_0.pdf")

def highlight_replacements(original, modified):
    """
    Compares original and modified text and prints replaced words.
    """
    diff = ndiff(original.split(), modified.split())
    replaced_words = [word[2:] for word in diff if word.startswith('- ')]
    
    if replaced_words:
        print("🔹 Words Replaced:", replaced_words)
    else:
        print("✅ No replacements found.")

for page_number in range(len(pdf_document)):
    page = pdf_document_1[page_number]  # Get a page
    text = page.get_text()  # Extract text

    print(f"🔹 Processing Page {page_number + 1}")

    original_text = text  # Keep the original text for comparison

    if page_number == 0:
        
        text = replace_names(text)
#         text = replace_dates(text)
#         text = replace_phone_numbers(text)
#         text = replace_addresses(text)
#         text = replace_companies(text)
#         text = replace_universities(text)
#         text = replace_links(text)
#         text = replace_emails(text)

    # ✅ Compare original vs. modified text
    highlight_replacements(original_text, text)

    print(f"Page {page_number + 1} Processed:\n{text}\n" + "-"*50)


🔹 Processing Page 1
🔹 Words Replaced: ['Andrea', 'Loizidou', 'Email:', 'Phone:', 'Location:', 'Miami,']
Page 1 Processed:
<NAME>
<NAME>: andrealoizidou@example.com | <NAME>: (555) 555-5555 | <NAME>: <NAME>, FL
Objective
Dedicated .NET Application Developer with over 4 years of experience in C#, .NET frameworks, and
full-stack development. Passionate about creating robust and scalable applications while
maintaining a focus on quality and efficiency. Excited to contribute to innovative solutions that
support educational institutions and student wellbeing.
Technical Skills
- Proficient in C#, ASP.NET, MVC, VB.NET, JavaScript, and jQuery
- Expertise in MSSQL Server, including stored procedures and SSIS packages
- Familiar with Visual Studio 2017/2022, Linq, HTML, CSS, Bootstrap, and Bootswatch
- Experienced in business analysis, stakeholder communication, and requirement documentation
- Strong background in full-stack web development and SQL query optimization
Professional Experience
.NET 

 ## NAMES

In [7]:
def replace_names(text):
    """
    Detects and replaces names with <NAME>.

    - Targets names appearing in the first 3-4 lines.
    - Ensures it does not replace company names, locations, or job titles.

    Args:
        text (str): The input resume text.

    Returns:
        str: The text with names replaced by <NAME>.
    """
    

#     # ✅ Target only the first 3-4 lines for name detection
#     lines = text.splitlines()[:2]  
#     name_candidates = []

#     # ✅ More restrictive regex for actual names (First and Last Name)
# #     name_pattern =r'\b[A-Z][a-z]+(?: [A-Z][a-z]+){0,1}\b'
#     name_pattern = r'\b[A-Z][a-z]+(?: [A-Z][a-z]+)?\b|\[\s*Your Name\s*\](?!\s*\w)'
    
#     for line in lines:
#         match = re.search(name_pattern, line)
#         if match:
#             name = match.group()
#             text = text.replace(name, '<NAME>', 1)
#             return text
#     return text
    # Split text into lines
    lines = text.split("\n")

    # Remove "Resume" if it's the first word in the first line
    if lines and re.match(r'^\s*Resume\b', lines[0], re.IGNORECASE):
        lines[0] = re.sub(r'^\s*Resume\b', '', lines[0]).strip()

    # Name pattern: Captures first and last name, optionally with a middle name or middle initial
    name_pattern = r'\b[A-Z][a-z]+(?: [A-Z]\.)?(?: [A-Z][a-z]+)?\b'

    # Check the first two lines for a name and replace it
    for i in range(min(2, len(lines))):  # Only check the first two lines
        lines[i] = re.sub(name_pattern, "<NAME>", lines[i])
    updated_text = "\n".join(lines)
    updated_text = re.sub(r'\byour name| John Doe\b', '<NAME>', updated_text, flags=re.IGNORECASE)
     

#     text =re.sub(r'\b[A-Z][a-z]+(?: [A-Z][a-z]+){0,1}\b',"<NAME>", text)
#     text = re.sub(r'\[Your Name\]', '<NAME>', text)

    return updated_text

text = """ Resume
Andrea A. Loizidou is the John Doe name of [your name] George Loizides
George Loizides
George Loizides
George Loizides
George Loizides
"""
print(replace_names(text))



<NAME> is the <NAME> name of [<NAME>] <NAME>
George Loizides
George Loizides
George Loizides
George Loizides



# A cool approach I can take is to delete everything before the name if it matches to == "resume" replace names with <name> and save the word you replaced, if you find a different one in the future then replace it again as <name>

## DATES

In [15]:
def replace_dates(text):
    """
    Replaces different date formats with <DATE>.
    
    Handles:
    - Full & Short month names followed by a year (e.g., "January 2020", "Jan 2020")
    - Numeric date formats (MM/DD/YYYY, DD-MM-YYYY, YYYY-MM-DD)
    - Month, Day, Year (e.g., "February 3, 2021")
    - Date ranges (e.g., "March 2015 - April 2020")
    - Standalone four-digit years (excluding phone numbers)
    - Context-based dates (e.g., "Graduated in 2020", "Worked for 3 years")

    Args:
        text (str): The input text containing dates.

    Returns:
        str: The text with dates replaced by <DATE>.
    """

    # 1️⃣ Full & Short Month Names + Year (e.g., "January 2020", "Jan 2020")
    text = re.sub(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept?|Oct|Nov|Dec)\.? \d{4}\b', '<DATE>', text)
    text = re.sub(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}\b', '<DATE>', text)

    # 2️⃣ Numeric Date Formats (MM/DD/YYYY, DD-MM-YYYY, YYYY-MM-DD)
    text = re.sub(r'\b\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4}\b', '<DATE>', text)  # Captures "12/31/2022", "31-12-2022"
    text = re.sub(r'\b\d{4}[-/.]\d{1,2}[-/.]\d{1,2}\b', '<DATE>', text)  # Captures "2022-12-31"

    # 3️⃣ Month, Day, Year Format (e.g., "February 3, 2021")
    text = re.sub(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}\b', '<DATE>', text)

    # 4️⃣ "From - To" Date Ranges (e.g., "March 2015 - April 2020")
    text = re.sub(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}\s?[-–]\s?(?:January|February|March|April|May|June|July|August|September|October|November|December)?\s?\d{4}\b', '<DATE>', text)

    # 5️⃣ Standalone Four-Digit Years (excluding phone numbers)
    text = re.sub(r'(?<!\d{3}[-.\s])\b\d{4}\b(?![-.\s]\d{3})', '<DATE>', text)
    
    text = re.sub(r'\(?\b\d{4}[-–]\d{4}\b\)?', '<DATE>', text)  # Handles (2008-2012) and 2008-2012

    #TODO: 10/20 - 11/22
    
    text = re.sub(r'Present', '<DATE>', text)
    text = re.sub(r'Current', '<DATE>', text)

    # 6️⃣ Context-Based Dates (e.g., "Graduated in 2020", "Worked for 3 years")
    text = re.sub(r'\b(?:Graduated|Completed|Finished|Started|Since|From) in \d{4}\b', '<DATE>', text)
    text = re.sub(r'\b(?:for|over|about) \d{1,2} years?(?: and \d{1,2} months?)?\b', '<DATE>', text)

    # 7️⃣ Decades (e.g., "The 1990s", "Early 2000s")
    text = re.sub(r'\b(?:the )?\d{4}s\b', '<DATE>', text)

    return text
text_samples = [
    "Worked from 2008-2012 as a software engineer.",
    "My education period was (2010-2014).",
    "Graduated in 2020.",
    "I worked from March 2015 - April 2020.",
    "The 1990s were a great time for tech.",
    "Currently working (Present)."
]
text = """
I was born on January 5, 1995. The project started on 12/10/2020 and ended on March 15, 2021.
She will visit on Monday, October 3rd. The contract expires in July 2024.
"""

# for sample in text_samples:
#     print(f"Original: {sample}")
#     print(f"Updated: {replace_dates(sample)}\n")
print(replace_dates(text))
    


I was born on <DATE>. The project started on <DATE> and ended on <DATE>.
She will visit on Monday, October 3rd. The contract expires in <DATE>.



## PHONE NUMBERS

In [9]:
def replace_phone_numbers(text):
    """
    Replaces all phone numbers and the exact string "[Your Phone Number]" with <PHONE_NUMBER>.

    Args:
        text (str): The input text.

    Returns:
        str: The text with phone numbers replaced by <PHONE_NUMBER>.
    """
    # ✅ Regex for real phone numbers
    phone_pattern = r'''
        (?:                                  # Non-capturing group for phone numbers
            (?:\+?\d{1,3}[-.\s]?)?           # Optional country code (+1, +44, etc.)
            (?:\(?\d{3}\)?[-.\s]?)           # Area code (with or without parentheses)
            \d{3}[-.\s]?\d{4}                # Main phone number (XXX-XXXX, XXX.XXXX, or XXX XXXX)
        )
    '''

    # ✅ First replace real phone numbers
    text = re.sub(phone_pattern, '<PHONE_NUMBER>', text, flags=re.VERBOSE)

    # ✅ Now separately replace the exact string "[Your Phone Number]"
    text = re.sub(r'\[Your Phone Number\]', '<PHONE_NUMBER>', text)

    return text

# # ✅ Test Cases
# text = """Call me at +1 (305) 555-1234 or (123) 456-7890.
# Also, my placeholder is [Your Phone Number].
# Another number: 987-654-3210."""

# print(replace_phone_numbers(text))


## ADDRESSES

In [11]:
def replace_addresses(text):
    """
    Detects and replaces addresses with <ADDRESS>.

    Handles:
    - "123 Main St."
    - "456 Elm Avenue"
    - "789 Oak Blvd, Miami, FL"
    - "321 5th Ave, NY"
    - "New York City, NY"
    - "Los Angeles, CA"
    - "San Francisco, California"
    
    Avoids:
    - False positives in general text
    
    Args:
        text (str): The input text containing addresses.

    Returns:
        str: The text with addresses replaced by <ADDRESS>.
    """

    # 1️⃣ Full Address Pattern (with Street Name & Number)
    address_pattern = r'\b\d{1,5}\s(?:[A-Za-z0-9]+\s){0,4}?(?:Ave|Avenue|St|Street|Blvd|Boulevard|Dr|Drive|Rd|Road|Ln|Lane|Way|Ct|Court|Pl|Place|Sq|Square|Hwy|Highway|Pkwy|Parkway|NW|NE|SW|SE)\.?\b(?:,\s?[A-Za-z\s]+,\s?[A-Z]{2})?'

    # 2️⃣ City + State Pattern (e.g., "Miami, FL", "New York, NY", "San Francisco, CA")
    city_state_pattern = r'\b(?:[A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\s?,\s?[A-Z]{2}\b'

    # Replace detected addresses
    text = re.sub(address_pattern, '<ADDRESS>', text)
    text = re.sub(city_state_pattern, '<ADDRESS>', text)
    text = re.sub(r'\[Your Address\]', '<ADDRESS>', text)
    
    return text

text = """
My home is at 504 Savannah River Drive, Summerville, SC 29485.
The package should be delivered to 1234 Maple Avenue, Springfield, 62704.
Meet me at 77 West 45th Street New York NY 10036.
Send it to 250 Elm St, Apt 4B, San Diego, CA 92101.
I work at 8801 Sunset Blvd, Los Angeles, CA.
The event is at 15 Oak Street, 30303.
I used to live at 55 Montgomery Rd., Cambridge, MA 02138.
Drop by at 22 Greenlake Drive, right next to the bakery, Seattle, WA 98103.
My friend lives at 742 Evergreen Terrace Springfield IL 62704.
The store is on 9812 Pine Ave, Dallas.
"""
print(replace_addresses(text))

#THE OTHER WAY IS BETTER!!!! THIS IS NOT VERY ACCURATE == NLP NER IS BETTER


My home is at <ADDRESS> 29485.
The package should be delivered to <ADDRESS>, Springfield, 62704.
Meet me at <ADDRESS> New York NY 10036.
Send it to <ADDRESS>, Apt 4B, <ADDRESS> 92101.
I work at <ADDRESS>.
The event is at <ADDRESS>, 30303.
I used to live at <ADDRESS>., <ADDRESS> 02138.
Drop by at <ADDRESS>, right next to the bakery, <ADDRESS> 98103.
My friend lives at 742 Evergreen Terrace Springfield IL 62704.
The store is on <ADDRESS>, Dallas.



## COMPANIES

In [13]:
def replace_companies(text):
    """
    Replaces only the company name after 'worked at', 'employer', 'worked for', 'experience' with <COMPANY>.

    Args:
        text (str): The input text.

    Returns:
        str: The text with company names replaced.
    """

    # ✅ Step 1: Replace company names in employment-related phrases
    company_pattern = r'(?i)\b(Worked at|Employer[:]?|Worked for[:]?)\s+([A-Z][a-zA-Z0-9&.()\-]+(?:\s[A-Z][a-zA-Z0-9&.()\-]+)*)'
    text = re.sub(company_pattern, r'\1 <COMPANY>', text)

    # ✅ Step 2: Replace full company names, including suffixes
    company_suffixes = r'LLC|Corp\.|Corporation|Solutions|Technologies|Enterprises|Group|Innovations|Services'

    # ✅ Matches full company names: "Facebook Technologies" → "<COMPANY>"
    pattern = rf'\b([A-Z][a-zA-Z0-9&.,()-]+(?:\s[A-Z][a-zA-Z0-9&.,()-]+)*)?\s*({company_suffixes})\b'

    def replace_match(match):
        return "<COMPANY>"

    text = re.sub(pattern, replace_match, text)

    return text

# ✅ Test Cases
text = """Worked at Shell Corporation as a Manager
Employer: Google LLC - Senior Engineer
Worked for Amazon Web Services (AWS)
Experience: Microsoft, 5 years
XYZ. Solutions
Facebook Technologies
"""
text1 = """
I recently had a meeting at Google and later visited Microsoft HQ. 
Apple is launching a new product next month. I also applied for a job at Goldman Sachs.
Amazon, Tesla, and Meta are investing in AI. The project is funded by OpenAI and IBM.
the organization i work for is XYZ LLC
"""

print(replace_companies(text1))



I recently had a meeting at Google and later visited Microsoft HQ. 
Apple is launching a new product next month. I also applied for a job at Goldman Sachs.
Amazon, Tesla, and Meta are investing in AI. The project is funded by OpenAI and IBM.
the organization i work for is <COMPANY>



## UNIVERSITIES

In [77]:
def replace_universities(text):
    """
    Detects and replaces university, college, and institute names with <INSTITUTION>.

    Handles:
    - "Graduated from Stanford University" → "Graduated from <INSTITUTION>"
    - "Studied at the University of California" → "Studied at <INSTITUTION>"
    - "Earned a degree from Harvard College" → "Earned a degree from <INSTITUTION>"
    - "Attended UCLA" → "Attended <INSTITUTION>"
    - "Massachusetts Institute of Technology" → "<INSTITUTION>"
    - "Graduated from Oxford Academy" → "<INSTITUTION>"
    
    Args:
        text (str): The input text containing university names.

    Returns:
        str: The text with universities replaced by <INSTITUTION>.
    """

    # ✅ Merged regex pattern for universities, colleges, institutes, polytechnics, academies
    university_pattern = r'\b(?:' + '|'.join([
        r'University of [A-Z][a-z]+(?:\s[A-Z][a-z]+)*',  # "University of Michigan"
        r'[A-Z][a-z]+ University',  # "Stanford University"
        r'[A-Z][a-z]+ Institute of Technology',  # "Massachusetts Institute of Technology"
        r'[A-Z][a-z]+ Institute',  # "California Institute"
        r'[A-Z][a-z]+ College',  # "Harvard College"
        r'[A-Z][a-z]+ Polytechnic',  # "XYZ Polytechnic"
        r'[A-Z][a-z]+ Academy',  # "Oxford Academy"
        r'[A-Z][a-z]+ School of [A-Z][a-z]+',  # "London School of Economics"
        r'UCLA|MIT|NYU|Caltech|Stanford|Harvard|Yale|Princeton|Oxford|Cambridge'  # Specific university abbreviations
    ]) + r')\b'

    # ✅ Replace detected university names
    text = re.sub(university_pattern, '<INSTITUTION>', text)

    return text


## LINKS

In [471]:
def replace_links(text):
    
    """
    Detects and replaces URLs and links with <LINK>, while avoiding emails and false positives.

    Handles:
    - "https://example.com"
    - "http://example.org/page"
    - "www.example.com"
    - "example.com/profile"
    - "github.com/johndoe"

    Avoids:
    - Emails like "john.doe@example.com"
    - Words containing ".com" that are not links (e.g., "E-commerce")

    Args:
        text (str): The input text containing links.

    Returns:
        str: The text with links replaced by <LINK>.
    """
    
    text = text.replace("unavailable)", "").replace("unavailable", "")


    # ✅ Regex for detecting valid links while avoiding emails
    link_pattern = r'(?<!\S)(https?://\S+|www\.\S+|\b(?!\S+@\S+)[a-zA-Z0-9.-]+\.com(?:/\S*)?)'

    # ✅ Replace detected links
    text = re.sub(link_pattern, '<LINK>', text)
    
    text = re.sub(r'\[Your LinkedIn Profile URL\]', '<LINK>', text)
    text = re.sub(r'\[(link unavailable)\]', '<LINK>', text)
    text = re.sub(r'\[link\]', '<LINK>', text)
    
    text = re.sub(r'\(\s*[^)]*\blink|Link\b[^)]*\s*\)', '<LINK>', text)
#     text = re.sub(r'\(\s*[^)]*(?i)\blink\b[^)]*\s*\)', '<LINK>', text)
                  
    return text
                  
text = """(link unavailable) Check out my profile (link provided here)." (Link)
    "Visit my portfolio at https://myportfolio.com."
    "(See the Link for details) andnd"
    "www.github.com/johndoe is my GitHub."
    "(this is a random sentence, ignore)"
    "[Your LinkedIn Profile URL] is missing."
    "Email me at john.doe@example.com (not a link)."
    "(click the link below for info)"""


print(replace_links(text))


<LINK> provided here)." (<LINK>
    "Visit my portfolio at <LINK>
    "(See the <LINK> andnd"
    "www.github.com/johndoe is my GitHub."
    "(this is a random sentence, ignore)"
    "<LINK> is missing."
    "Email me at john.doe@example.com <LINK>)."
    "<LINK> below for info)


In [16]:
import re

def replace_links(text):
    """
    Detects and replaces valid links while avoiding emails and other non-URL text.
    """

    # ✅ Step 1: Remove "unavailable" placeholders
    text = text.replace("unavailable)", "").replace("unavailable", "")

    # ✅ Step 2: Regex for detecting valid links while avoiding emails
    link_pattern = r'''
        (?<!\S)  # Ensure it's not part of another word
        (       
            https?://\S+  |    # Matches "http://example.com", "https://..."
            www\.\S+       |    # Matches "www.example.com"
            \b(?!\S+@\S+)[a-zA-Z0-9.-]+\.com(?:/\S*)?  # Matches "example.com", avoiding emails
        )
    '''

    # ✅ Step 3: Replace detected links
    text = re.sub(link_pattern, '<LINK>', text, flags=re.VERBOSE)

    # ✅ Step 4: Replace placeholders like "[Your LinkedIn Profile URL]" and variations of "(link)"
    placeholder_patterns = [
        r'\[Your LinkedIn Profile URL\]',  
        r'\[link unavailable\]',  
        r'\[link\]',  
        r'\(\s*[^)]*\blink\b[^)]*\s*\)'  # Matches variations like "(See the Link for details)"
    ]

    for pattern in placeholder_patterns:
        text = re.sub(pattern, '<LINK>', text, flags=re.IGNORECASE)

    return text

# ✅ Test Cases
text = """(link unavailable) Check out my profile (link provided here)." (Link)
"Visit my portfolio at https://myportfolio.com."
"www.github.com/johndoe is my GitHub."
"(See the Link for details)"
"[Your LinkedIn Profile URL] is missing."
"Email me at john.doe@example.com (not a link)."
"(click the link below for info)"
"Check my website at example.com/about."
"""

print(replace_links(text))


<LINK>." <LINK>
"Visit my portfolio at <LINK>
"www.github.com/johndoe is my GitHub."
"<LINK>"
"<LINK> is missing."
"Email me at john.doe@example.com <LINK>."
"<LINK>"
"Check my website at <LINK>



## EMAILS

In [204]:
def replace_emails(text):
    """
    Replaces all email addresses in the text with <EMAIL>.

    Args:
        text (str): The input text.

    Returns:
        str: The text with email addresses replaced by <EMAIL>.
    """
    email_pattern = r'\S+@\S+'  # Matches any text containing "@"
    text = re.sub(email_pattern, '<EMAIL>', text)
    text = re.sub(r'\Your Email Address', '<EMAIL>', text)
    return text


In [479]:
import os
import re
import textract

def scan_for_unreplaced_addresses(folder):
    """
    Scans all text files in the given folder and checks if any addresses remain unreplaced
    after applying `replace_addresses`. If any file contains an address, its name is printed.

    Args:
        folder (str): Path to the folder containing cleaned resumes.
    """
    flagged_files = []

    # Process each file in the cleaned folder
    for root, dirs, files in os.walk(folder):
        for filename in files:
            if not filename.endswith(".txt"):  # Only check text files
                continue

            filepath = os.path.join(root, filename)

            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    raw_text = f.read()
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                continue

            # Apply the address replacement function
            cleaned_text = replace_addresses(raw_text)

            # If the cleaned text still contains an address, flag the file
            if raw_text != cleaned_text:  # If replacement occurred, continue
                continue

            # If no replacement happened, store the file path
            flagged_files.append(filepath)

    # Output the flagged files
    if flagged_files:
        print(f"\nTotal files with unreplaced addresses: {len(flagged_files)}")
        print("\nFiles containing unreplaced addresses:")
        for file in flagged_files:
            print(file)
    else:
        print("All addresses have been replaced successfully.")

# Run the scanning function
scan_for_unreplaced_addresses(cleaned_folder)





Total files with unreplaced addresses: 858

Files containing unreplaced addresses:
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/105_resume_job2_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/78_resume_job4_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/31_resume_job1_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/142_resume_job2_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/66_resume_job5_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/21_resume_job5_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0/46_resume_job3_authentic_0_cleaned.txt
/Users/andrealoizidou/Desktop/capstone project/authentic_res

## Deleting all Files for Reprocessing Purposes. 

In [357]:
import shutil
import os

# Define the cleaned folder path
cleaned_folder = "/Users/andrealoizidou/Desktop/capstone project/data_generation/cleaned_resumes_0"

# Delete all files in the cleaned folder
if os.path.exists(cleaned_folder):
    shutil.rmtree(cleaned_folder)

# Recreate the cleaned folder
os.makedirs(cleaned_folder, exist_ok=True)
print("Cleaned folder reset. Ready to reprocess resumes.")

Cleaned folder reset. Ready to reprocess resumes.


## Function to Clean Text

In [375]:
def clean_text(text, remove_stopwords=False):
    text = replace_names(text)
    text = replace_dates(text)
    text = replace_phone_numbers(text)
    text = replace_addresses(text)
    text = replace_companies(text)
    text = replace_universities(text)
    text = replace_links(text)
    text = replace_emails(text)
    return text



## Code to Process the cleaned resumes

In [371]:
import os
import textract
import shutil

# AI CLEANED RESUMES (1)

In [453]:
original_folder = "/Users/andrealoizidou/Desktop/capstone project/data_generation"
cleaned_folder = os.path.join(original_folder, "cleaned_resumes_1")

# Ensure the cleaned folder exists
os.makedirs(cleaned_folder, exist_ok=True)
print(f"Cleaned resumes will be saved in: {cleaned_folder}")


overwrite_cleaned = True  # Set to True to overwrite existing cleaned files

# Process each resume
for root, dirs, files in os.walk(original_folder):
    # Skip the `cleaned_resumes_1` folder
    if root.startswith(cleaned_folder):
        continue

    for filename in files:
        # Skip non-resume files
        if not filename.endswith((".pdf", ".docx", ".txt")):
            continue

        # Extract text from the resume
        filepath = os.path.join(root, filename)
        try:
            raw_text = textract.process(filepath).decode("utf-8")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

        # Clean the text
        cleaned_text = clean_text(raw_text, remove_stopwords=False)

        # Define cleaned filename
        if "_cleaned" not in filename:
            cleaned_filename = os.path.splitext(filename)[0] + "_cleaned.txt"
        else:
            cleaned_filename = filename  # Keep the original cleaned filename

        cleaned_filepath = os.path.join(cleaned_folder, cleaned_filename)

        # Handle overwriting logic
        if os.path.exists(cleaned_filepath) and not overwrite_cleaned:
            print(f"Skipped (already cleaned): {cleaned_filename}")
            continue

        # Write cleaned text to file
        with open(cleaned_filepath, "w", encoding="utf-8") as f:
            f.write(cleaned_text)
        print(f"Processed and saved: {cleaned_filename}")

print("All resumes have been cleaned and saved!")

Cleaned resumes will be saved in: /Users/andrealoizidou/Desktop/capstone project/data_generation/cleaned_resumes_1
Processed and saved: 50_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 27_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 14_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 6_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 45_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 32_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 35_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 42_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 9_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 1_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 13_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 28_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 20_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 25_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 4_resume_job4_Gemini_1_cleaned.txt
Processed and saved: 16_resume_job

# AUTHENTIC - CLEANED RESUMES (0)

In [454]:
original_folder = "/Users/andrealoizidou/Desktop/capstone project/authentic_resumes"
cleaned_folder = os.path.join(original_folder, "cleaned_resumes_0")

# Ensure the cleaned folder exists
os.makedirs(cleaned_folder, exist_ok=True)
print(f"Cleaned resumes will be saved in: {cleaned_folder}")


overwrite_cleaned = True  # Set to True to overwrite existing cleaned files

# Process each resume
for root, dirs, files in os.walk(original_folder):
    # Skip the `cleaned_resumes_1` folder
    if root.startswith(cleaned_folder):
        continue

    for filename in files:
        # Skip non-resume files
        if not filename.endswith((".pdf", ".docx", ".txt")):
            continue

        # Extract text from the resume
        filepath = os.path.join(root, filename)
        try:
            raw_text = textract.process(filepath).decode("utf-8")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

        # Clean the text
        cleaned_text = clean_text(raw_text, remove_stopwords=False)

        # Define cleaned filename
        if "_cleaned" not in filename:
            cleaned_filename = os.path.splitext(filename)[0] + "_cleaned.txt"
        else:
            cleaned_filename = filename  # Keep the original cleaned filename

        cleaned_filepath = os.path.join(cleaned_folder, cleaned_filename)

        # Handle overwriting logic
        if os.path.exists(cleaned_filepath) and not overwrite_cleaned:
            print(f"Skipped (already cleaned): {cleaned_filename}")
            continue

        # Write cleaned text to file
        with open(cleaned_filepath, "w", encoding="utf-8") as f:
            f.write(cleaned_text)
        print(f"Processed and saved: {cleaned_filename}")

print("All resumes have been cleaned and saved!")

Cleaned resumes will be saved in: /Users/andrealoizidou/Desktop/capstone project/authentic_resumes/cleaned_resumes_0
Processed and saved: 139_resume_job4_authentic_0_cleaned.txt
Processed and saved: 51_resume_job4_authentic_0_cleaned.txt
Processed and saved: 33_resume_job4_authentic_0_cleaned.txt
Processed and saved: 74_resume_job4_authentic_0_cleaned.txt
Processed and saved: 188_resume_job4_authentic_0_cleaned.txt
Processed and saved: 106_resume_job4_authentic_0_cleaned.txt
Processed and saved: 25_resume_job4_authentic_0_cleaned.txt
Processed and saved: 82_resume_job4_authentic_0_cleaned.txt
Processed and saved: 141_resume_job4_authentic_0_cleaned.txt
Processed and saved: 152_resume_job4_authentic_0_cleaned.txt
Processed and saved: 91_resume_job4_authentic_0_cleaned.txt
Processed and saved: 36_resume_job4_authentic_0_cleaned.txt
Processed and saved: 115_resume_job4_authentic_0_cleaned.txt
Processed and saved: 121_resume_job4_authentic_0_cleaned.txt
Processed and saved: 67_resume_job4_